In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.utils import shuffle

## Row-wise processing

In [3]:
#merging and normalizing outputs of the PCL to the AE format
#PCL -> AE
def load_and_normalize(folder_path):
    all_data = []
    min_val, max_val = -10, 7.5
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".npy"):
            file_path = os.path.join(folder_path, file_name)
            data = np.load(file_path)
            
            # Normalize each line
            for line in data:
                
                line = np.maximum(line, min_val)
                line = np.minimum(line, max_val)
                line = (line - min_val) / (max_val - min_val)
                all_data.append(line)
    
    # Concatenate all data
    all_data = np.vstack(all_data)
    
    return all_data

folder_path = '/home/carol/NaroNet-main/NaroNet-main/Endometrial_POLE/Patch_Contrastive_Learning/Image_Patch_Representation'
allembs = load_and_normalize(folder_path)
print(np.shape(allembs))
np.save('ConcInp.npy', allembs)

(3847875, 265)


In [ ]:
#splitting and renormalzie AE output to the Naronet format
#AE->Naronet
init_embs_path = "/home/carol/NaroNet-main/NaroNet-main/Endometrial_POLE/Patch_Contrastive_Learning/Image_Patch_Representation"
rec_path = "/home/carol/NaroNet-main/EXP3/IDEC-toy/results/embs/reconstructed_x.npy"
out_path = "/home/carol/NaroNet-main/EXP3/Data/Split reconstruction"
reconstructed_embeddings = np.load(rec_path)
reconstructed_embeddings = reconstructed_embeddings * (7.5 - (-10)) + (-10) #rescale to the original value range
current_position= 0
for file_name in os.listdir(init_embs_path):
    file_path = os.path.join(init_embs_path, file_name)
    cr_emb = np.load(file_path)  
    line_count = cr_emb.shape[0]
    
    extracted_embeddings = reconstructed_embeddings[current_position : current_position + line_count]
    current_position += line_count
    new_file_path = os.path.join(out_path, file_name)
    np.save(new_file_path, extracted_embeddings)

## Column-wise processing

In [5]:
#merging outputs of the PCL to the AE format
#PCL -> AE
def load(folder_path):
    all_data = []
    min_val, max_val = -10, 7.5
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".npy"):
            file_path = os.path.join(folder_path, file_name)
            data = np.load(file_path)


            for line in data:
                all_data.append(line)
    
    # Concatenate all data
    all_data = np.vstack(all_data)
    
    return all_data

folder_path = '/home/carol/NaroNet-main/NaroNet-main/Endometrial_POLE/Patch_Contrastive_Learning/Image_Patch_Representation'
allembs = load(folder_path)
print(np.shape(allembs))
np.save('ConcInp_noNorm.npy', allembs)

(3847875, 265)


In [1]:
def normal(data):
    # Assuming `data` is your (300k, 256) NumPy array
    data_min = data.min(axis=0)  # Minimum value of each column
    data_max = data.max(axis=0)  # Maximum value of each column

    # Min-Max normalization to scale data to the [0, 1] range
    data_normalized = (data - data_min) / (data_max - data_min)

    # Handle potential divisions by zero if a column is constant
    data_normalized = np.nan_to_num(data_normalized, nan=0.0)
    return data_normalized

In [4]:
data = np.load("/home/carol/NaroNet-main/EXP3/ConcInp_noNorm.npy")
normal_data  = normal(data)
np.save('ConcInpCol.npy', normal_data)

Rescale

In [1]:
def rescale_col(data_normalized):
    # Assuming `data_normalized` is your normalized data array
    # Define min and max values for the rescaling process
    min_val_first_two = -10
    max_val_first_two = 1750
    min_val_rest = -10
    max_val_rest = 7.5

    # Initialize an array to hold the rescaled values, same shape as `data_normalized`
    data_rescaled = np.zeros_like(data_normalized)

    # Rescale the first two columns
    for i in range(2):
        data_rescaled[:, i] = data_normalized[:, i] * (max_val_first_two - min_val_first_two) + min_val_first_two
        # Cap values at 1750 for the first two columns
        data_rescaled[:, i] = np.where(data_rescaled[:, i] > max_val_first_two, max_val_first_two, data_rescaled[:, i])

    # Rescale the rest of the columns
    for i in range(2, data_normalized.shape[1]):
        data_rescaled[:, i] = data_normalized[:, i] * (max_val_rest - min_val_rest) + min_val_rest
        # No specific capping mentioned for these, but you could apply similar logic if needed

    # Now `data_rescaled` contains the rescaled values, with the first two columns capped at 1750
    return data_rescaled

In [5]:
#splitting and renormalzie AE output to the Naronet format
#AE->Naronet
#Image patch Rep needs to be non-empty
init_embs_path = "/home/carol/NaroNet-main/NaroNet-main/Endometrial_POLE/Patch_Contrastive_Learning/Image_Patch_Representation"
rec_path = "/home/carol/NaroNet-main/EXP3/IDEC-toy/results/embs/reconstructed_x.npy"
out_path = "/home/carol/NaroNet-main/EXP3/Data/Split reconstruction"
reconstructed_embeddings = np.load(rec_path)
reconstructed_embeddings = rescale_col(reconstructed_embeddings) #rescale to the original value range
current_position= 0
for file_name in os.listdir(init_embs_path):
    file_path = os.path.join(init_embs_path, file_name)
    cr_emb = np.load(file_path)  
    line_count = cr_emb.shape[0]
    
    extracted_embeddings = reconstructed_embeddings[current_position : current_position + line_count]
    current_position += line_count
    new_file_path = os.path.join(out_path, file_name)
    np.save(new_file_path, extracted_embeddings)

In [9]:
import pandas as pd

def compare_labels(path1, path2):
    # Load the Excel files
    data1 = pd.read_excel(path1, usecols=[1, 3], header=None, names=['Subject_name', 'Label'])
    data2 = pd.read_excel(path2, usecols=[1, 3], header=None, names=['Subject_name', 'Label'])
    
    # Merge the two datasets on 'Subject_name'
    merged_data = pd.merge(data1, data2, on='Subject_name', suffixes=('_1', '_2'))
    
    # Find mismatches in 'Label'
    mismatches = merged_data[merged_data['Label_1'] != merged_data['Label_2']]
    
    # Print the subject names with differing labels
    for subject in mismatches['Subject_name']:
        print(subject)
    

# Example usage:
# compare_labels('path_to_file1.xlsx', 'path_to_file2.xlsx')
path1 = "/home/carol/NaroNet-main/Datasets/Results/11_Exp2_Repeat1/NaroNet/Response/Cross_validation_results/Prediction_values_Response_Fold13.xlsx"
path2 = "/home/carol/NaroNet-main/Datasets/Results/10_Exp2_CorFolds/NaroNet/Response/Cross_validation_results/Prediction_values_Response_Fold13.xlsx"
compare_labels(path1,path2)